In [1]:
# 라이브러리 호출

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = pd.read_csv('../file/train_add_category.csv')
df = df.drop(['Unnamed: 0','predicted'], axis=1)
df

data = df.drop('Pawpularity', axis=1)
target = df['Pawpularity']

data['category'].replace('dog',0, inplace=True)
data['category'].replace('cat',1, inplace=True)
# data.drop('Id', axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.25, random_state=0)

In [131]:
# pd.DataFrame(X_test.iloc[0])
# X_test.iloc[0][1:]
np.array(X_train.loc[X_train['Id']=="4900e463696cc862724c3aa96b5d7487"].iloc[:,1:])
# df.loc[df['Id']=="4900e463696cc862724c3aa96b5d7487"].iloc[0][13]

X_train.loc[X_train['Id']=="4900e463696cc862724c3aa96b5d7487"].iloc[:,1:]

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,category
2838,0,1,1,1,0,0,0,0,0,0,0,0,0


In [2]:
# 코사인 유사도를 계산하는 함수
def cos_sim(a,b):
    return np.dot(a,b) / (np.linalg.norm(a)*(np.linalg.norm(b)))

In [106]:
# x_train = X_train.drop('Id', axis=1)
# x_test = X_test.drop('Id', axis=1)

# 코사인 유사도를 구하는 반복문
sim_score = []
for i in range (len(X_train)):
    name = X_train.iloc[i]['Id']
    if name != X_test.iloc[0]['Id']:
        sim_score.append({
            'Id':name,
            'score':cos_sim(np.array(X_test.iloc[0][1:]), np.array(X_train.loc[X_train['Id']==name].iloc[:,1:])[0]),
            'Pawpularity' : df.loc[df['Id']==name].iloc[0][13]
        })
        
# print('* 유사도 순위')
# sim_score_df = pd.DataFrame(sim_score)
# sim_score_df.head(5)

<ipython-input-51-3a0cb5f5d8f0>:2: RuntimeWarning: invalid value encountered in true_divide
  return np.dot(a,b) / (np.linalg.norm(a)*(np.linalg.norm(b)))


In [107]:
# 데이터프레임으로 만들기
print('* 유사도 순위')
#코사인 유사도가 높은 순서대로 정렬
sim_score = sorted(sim_score, key=lambda x:x['score'], reverse=True)
sim_score_df = pd.DataFrame(sim_score)
# 코사인 유사도가 1인것 출력
sim_score_df.loc[sim_score_df['score']==1]

* 유사도 순위


,Id,score,Pawpularity
0,535893d095e7e378373e329032787805,1.0,42
1,8ddb21bcb5d2d2a5402998e8ec6250fc,1.0,24
2,afa5c11060ad0523919e43138c5d3294,1.0,34
3,14e722a86306e100874bdc0f2b40270a,1.0,27
4,43de6e8160f6be22f767e9a9e043bbc3,1.0,31
...,...,...,...
7311,d3df4e2dc7713406b258c4e8b5c935cc,1.0,55
7312,726f589971edff773dd3dfd9303d4aa3,1.0,35
7313,7c5bb869ebdd55911ba1cee387e6861a,1.0,21
7314,6d3bd371a92d76ba0b090b40c93a37a3,1.0,22


In [109]:
# 코사인 유사도가 1인 'id'의 조회수 평균
sim_score_df['Pawpularity'].loc[sim_score_df['score']==1].mean()

37.361870503597125

In [4]:
import time

start = time.time()
pawpularity_pred =[]
for j in range(len(X_test)):
    sim_score = []
    for i in range (len(X_train)):
        name = X_train.iloc[i]['Id']
        if name != X_test.iloc[0]['Id']:
            sim_score.append({
                'Id':name,
                'score':cos_sim(np.array(X_test.iloc[j][1:]), np.array(X_train.loc[X_train['Id']==name].iloc[:,1:])[0]),
                'Pawpularity' : df.loc[df['Id']==name].iloc[0][13]
            })
    #코사인 유사도가 높은 순서대로 정렬
    sim_score = sorted(sim_score, key=lambda x:x['score'], reverse=True)
    sim_score_df = pd.DataFrame(sim_score)
    pawpularity_pred.append(sim_score_df['Pawpularity'].loc[sim_score_df['score']==1].mean())
            
            
print('take time : ', time.time()-start)

<ipython-input-2-99e50f0294b4>:3: RuntimeWarning: invalid value encountered in true_divide
  return np.dot(a,b) / (np.linalg.norm(a)*(np.linalg.norm(b)))


take time :  41141.903379678726


In [13]:
# from sklearn.metrics import mean_squared_error

# mse = mean_squared_error(y_test, pawpularity_pred)
# rmse=np.sqrt(mse)

# print('RMSE : {0:2f}'.format(rmse))

In [34]:
sim_score = sorted(sim_score, key=lambda x:x['score'], reverse=True)
sim_score_df = pd.DataFrame(sim_score)
sim_score_df
sim_score

[{'Id': '7991e8647e0e3335b629d89c4f07cfdb',
  'score': 0.9999999999999998,
  'Pawpularity': 23},
 {'Id': 'd490235bb57e77c16965ecbccb0908e9',
  'score': 0.9999999999999998,
  'Pawpularity': 29},
 {'Id': 'b088361fa4dd8f112b5d1d10ea5ed208',
  'score': 0.8164965809277259,
  'Pawpularity': 62},
 {'Id': '76e3988dd37d391f24104285a177b30f',
  'score': 0.8164965809277259,
  'Pawpularity': 34},
 {'Id': '00b151a572c9aabedf8cfce0fa18be25',
  'score': 0.8164965809277259,
  'Pawpularity': 21},
 {'Id': '36ff9acbce2c0791644f445dc1d5778e',
  'score': 0.8164965809277259,
  'Pawpularity': 20},
 {'Id': '03f9353072191f8877825f0c7eefbea7',
  'score': 0.8164965809277259,
  'Pawpularity': 50},
 {'Id': '5f1dfc3290b10cb22cf2fb16d79ef95d',
  'score': 0.8164965809277259,
  'Pawpularity': 19},
 {'Id': 'fa6894b340185fa33b9542cc13e8d856',
  'score': 0.8164965809277259,
  'Pawpularity': 21},
 {'Id': '063d79b149f4d163eae86f777a39a42f',
  'score': 0.8164965809277259,
  'Pawpularity': 100},
 {'Id': '5264b82edc283a682781

In [14]:
# pawpularity_pred

반복문을 짜는 과정에서 잘 못짠 코드 있음 수정해서 submission 제출 필요